In [1]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score, roc_auc_score

# from pykospacing import Spacing
# import re

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
train_df = pd.read_csv(r'./data/korean-hate-speech-detection/train.hate.csv')

train_text = open(r'./data/korean-hate-speech-detection/train.news_title.txt', 'r')
train_texts = train_text.readlines()

train_text_list = []
for line in train_texts:
    train_text_list.append(line)
train_text.close()

train_df['text'] = train_text_list
train_df

In [ ]:
lengh = []
for i in train_df['comments']:
    lengh.append(len(i))
max(lengh)

In [54]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()  # 소문자 변경
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 기본 불용어 불러오기
korean_stopwords_path = "data/stopwords-ko.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 불용어 처리 함수 수정
def remove_stopwords(text, stopwords):
    words = text.split()  # 문장을 단어로 분리
    filtered_text = [word for word in words if word not in stopwords]  # 단어가 불용어 목록에 없는 경우만 추가
    return ' '.join(filtered_text)  # 필터링된 단어들 다시 조합

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(train_df))):
    comment_text = train_df.loc[i, 'comments']
    newstile_text = train_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    train_df.loc[i, 'processed_comments'] = cleaned_comment
    train_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [ ]:
train_df.loc[train_df['label'] == 'none', 'labels'] = 0
train_df.loc[train_df['label'] == 'offensive', 'labels'] = 1
train_df.loc[train_df['label'] == 'hate', 'labels'] = 2

train_df['labels'] = train_df['labels'].astype(int)
train_df.info()

In [70]:
train_df.to_csv('./data/train_df_processed.csv', index=False, encoding='utf-8-sig')

In [ ]:
dev_df = pd.read_csv(r'./data/korean-hate-speech-detection/dev.hate.csv')

dev_text = open(r'./data/korean-hate-speech-detection/dev.news_title.txt', 'r')
dev_texts = dev_text.readlines()

dev_text_list = []
for line in dev_texts:
    dev_text_list.append(line)
dev_text.close

dev_df['text'] = dev_text_list
dev_df

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(dev_df))):
    comment_text = dev_df.loc[i, 'comments']
    newstile_text = dev_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    dev_df.loc[i, 'processed_comments'] = cleaned_comment
    dev_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [88]:
dev_df.loc[dev_df['label'] == 'none', 'labels'] = 0
dev_df.loc[dev_df['label'] == 'offensive', 'labels'] = 1
dev_df.loc[dev_df['label'] == 'hate', 'labels'] = 2
dev_df['labels'] = dev_df['labels'].astype(int)

dev_df.to_csv('./data/dev_df_processed.csv', index=False, encoding='utf-8-sig')

In [ ]:
dev_df = pd.read_csv('./data/dev_df_processed.csv')
dev_df

In [ ]:
test_df = pd.read_csv(r'./data/korean-hate-speech-detection/test.hate.no_label.csv')

test_text = open(r'./data/korean-hate-speech-detection/test.news_title.txt')
test_texts = test_text.readlines()
test_text_list = []
for line in test_texts:
    test_text_list.append(line)

test_df['text'] = test_text_list
test_df

In [ ]:
# 문장 전처리 및 불용어 처리
for i in tqdm(range(len(test_df))):
    comment_text = test_df.loc[i, 'comments']
    newstile_text = test_df.loc[i, 'text']
    processed_comment = preprocessing(comment_text)
    processed_newstile = preprocessing(newstile_text)
    cleaned_comment = remove_stopwords(processed_comment, stopwords)
    cleaned_newstile = remove_stopwords(processed_newstile, stopwords)
    test_df.loc[i, 'processed_comments'] = cleaned_comment
    test_df.loc[i, 'processed_newstitle'] = cleaned_newstile

In [67]:
test_df.to_csv('./data/test_df_processed.csv', index=False, encoding='utf-8-sig')

In [2]:
train_df = pd.read_csv(r'./data/train_df_processed.csv')
dev_df = pd.read_csv(r'./data/dev_df_processed.csv')
test_df = pd.read_csv(r'./data/test_df_processed.csv')

In [3]:
model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/tfGPU/lib/python3.9/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/p

In [5]:
import torch

# GPU 설정
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print("device:", device)

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print("device:", device)

model.to(device)

device: mps


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [95]:
# # 데이터 토크나이징

# tokenized_train = tokenizer(
#     list(train_df['processed_comments']),
#     return_tensors='pt',
#     max_length=128,
#     padding=True,
#     truncation=True,          # max_length 초과 토큰 truncate
#     add_special_tokens=True,  # special token 추가
# )

# tokenized_dev = tokenizer(
#     list(dev_df['processed_comments']),
#     return_tensors='pt',
#     max_length=128,
#     padding=True,
#     truncation=True,          # max_length 초과 토큰 truncate
#     add_special_tokens=True,  # special token 추가
# )

In [102]:
# # 데이터 셋 설정
# class CurseDataSet(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item
    
#     def __len__(self):
#         return len(self.labels)
    
# train_label = train_df['labels'].values
# dev_label = dev_df['labels'].values

# train_dataset = CurseDataSet(tokenized_train, train_label)
# dev_dataset = CurseDataSet(tokenized_dev, dev_label)

In [6]:
# KF_DeBERT모델 텍스트 토큰화
train_encodings = tokenizer(train_df['processed_comments'].tolist(), truncation=True, padding=True)  # truncation=최대길이를 초과하는 것에 대한 처리
train_df['encoding'] = train_encodings
dev_encodings = tokenizer(dev_df['processed_comments'].tolist(), truncation=True, padding=True)  # truncation=최대길이를 초과하는 것에 대한 처리
dev_df['encoding'] = dev_encodings

# 데이터셋 준비
train_df['input_ids'] = train_encodings['input_ids']  # input_ids=문장을 토크나이저가 일정 단위로 쪼개서 vocab에 들어있는 숫자로 치환한 것
train_df['attention_mask'] = train_encodings['attention_mask']  # attention_mask=attention대상인지 아닌지를 나타냄. 대상이면1, 아니면0, 패딩된건 0으로 학습대상x
dev_df['input_ids'] = dev_encodings['input_ids']  # input_ids=문장을 토크나이저가 일정 단위로 쪼개서 vocab에 들어있는 숫자로 치환한 것
dev_df['attention_mask'] = dev_encodings['attention_mask']  # attention_mask=attention대상인지 아닌지를 나타냄. 대상이면1, 아니면0, 패딩된건 0으로 학습대상x

X_train = train_df[['input_ids', 'attention_mask']]
y_train = train_df['labels']
X_test = dev_df[['input_ids', 'attention_mask']]
y_test = dev_df['labels']

In [7]:
train_dataset = Dataset.from_pandas(X_train.join(y_train))
dev_dataset = Dataset.from_pandas(X_test.join(y_test))

In [8]:
# # 하이퍼파라미터 옵션 설정
param_grid = {
    'learning_rate': [5e-5],  # 2e-5, 3e-5, 5e-5
    'per_device_train_batch_size': [8],
    'num_train_epochs': [3]
}

# param_grid = {
#     'learning_rate': [3e-5, 2e-5, 5e-5],  # 학습률
#     'per_device_train_batch_size': [4, 8, 16, 32],  # 베치크기
#     'num_train_epochs': [2, 4, 6, 8]  # 에포크 수
# }

In [9]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 수치 초기화
best_accuracy = 0
best_F1 = 0
best_params = None
results = []

for learning_rate in tqdm(param_grid['learning_rate']):
    for batch_size in param_grid['per_device_train_batch_size']:
        for epochs in param_grid['num_train_epochs']:
            try:
                if batch_size > len(train_dataset):
                    print(f"Skipping batch size {batch_size} as it is larger than the dataset size {len(train_dataset)}")
                    continue

                # TrainingArguments 설정
                training_args = TrainingArguments(
                    output_dir="./results",  # 학습된 모델과 결과를 저장할 경로 설정
                    run_name="my_experiment",  # 고유한 run_name 설정  // 로그인 피하기위해
                    evaluation_strategy="epoch",  # 각 에포크마다 평가 수행
                    learning_rate=learning_rate,  # 학습률 설정
                    per_device_train_batch_size=batch_size,  # 학습 배치 크기 설정
                    per_device_eval_batch_size=batch_size,  # 평가 배치 크기 설정
                    num_train_epochs=epochs,  # 현재 학습 에포크 수 설정
                    weight_decay=0.01,  # 가중치 감쇠 설정
                    logging_dir='./logs',  # 로그 저장 경로 설정
                    logging_steps=10,  # 로그를 기록할 단계 수 설정
                )

                # 모델을 학습 모드로 설정
                model.train()

                # Trainer 생성
                trainer = Trainer(
                    # model=model, #.to(),
                    model=model.to(device),  # 모델을 GPU로 이동
                    args=training_args,
                    train_dataset=train_dataset,  # 훈련 데이터셋
                    eval_dataset=dev_dataset,  # 평가 데이터셋
                )

                # 모델 학습
                trainer.train()

                # 훈련 데이터에서 평가
                train_results = trainer.predict(train_dataset)
                train_preds = np.argmax(train_results.predictions, axis=1)
                train_labels = train_results.label_ids
                train_accuracy = accuracy_score(train_labels, train_preds)
                train_precision = precision_score(train_labels, train_preds, average='binary')
                train_recall = recall_score(train_labels, train_preds, average='binary')
                train_f1 = f1_score(train_labels, train_preds, average='binary')
                train_loss = train_results.metrics['eval_loss'] if 'eval_loss' in train_results.metrics else None

                # 테스트 데이터에서 평가
                eval_results = trainer.evaluate()
                predictions = trainer.predict(dev_dataset)
                preds = np.argmax(predictions.predictions, axis=1)
                labels = predictions.label_ids
                eval_accuracy = accuracy_score(labels, preds)
                eval_precision = precision_score(labels, preds, average='binary')
                eval_recall = recall_score(labels, preds, average='binary')
                eval_f1 = f1_score(labels, preds, average='binary')
                eval_loss = eval_results['eval_loss'] if 'eval_loss' in eval_results else None
                tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
                eval_specificity = tn / (tn + fp)

                # 성능 메트릭 계산
                accuracy = accuracy_score(labels, preds)
                precision = precision_score(labels, preds, average='binary')
                recall = recall_score(labels, preds, average='binary')
                f1 = f1_score(labels, preds, average='binary')
                tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
                specificity = tn / (tn + fp)

                # 결과 저장
                results.append({
                    'learning_rate': learning_rate,
                    'batch_size': batch_size,
                    'num_train_epochs': epochs,
                    'train_accuracy': train_accuracy,
                    'train_precision': train_precision,
                    'train_recall': train_recall,
                    'train_f1': train_f1,
                    'train_loss': train_loss,
                    'eval_accuracy': eval_accuracy,
                    'eval_precision': eval_precision,
                    'eval_recall': eval_recall,
                    'eval_f1': eval_f1,
                    'eval_loss': eval_loss,
                    'eval_specificity': eval_specificity
                })

                # 최고 성능 모델 기록
                if eval_accuracy > best_accuracy:
                    best_accuracy = eval_accuracy
                    best_F1 = eval_f1
                    best_params = {
                        'learning_rate': learning_rate,
                        'batch_size': batch_size,
                        'num_train_epochs': epochs
                    }

            except Exception as e:
                print(f"Error with parameters: learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
                print(f"Exception: {e}")
                continue

# 최적의 하이퍼파라미터 조합 출력
print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)
print("Best F1:", best_F1)

100%|██████████| 1/1 [00:00<00:00, 76.85it/s]

Error with parameters: learning_rate=5e-05, batch_size=8, epochs=3
Exception: __init__() got an unexpected keyword argument 'dispatch_batches'
Best Parameters: None
Best Accuracy: 0
Best F1: 0
